In [26]:
import re

from itertools import cycle, combinations, permutations, tee
from collections import Counter, defaultdict, deque
from io import StringIO
from functools import reduce
from pprint import pprint

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def read_input(day, fn=str.strip):
    """//
    Return a list of the input lines mapped by fn
    
    example: 
    >>> read_input('01', int)  # read input file, map all lines to int
    
    Inspired by Peter Norvig: https://github.com/norvig/pytudes
    
    """
    return list(map(fn, open(f'input\{day}.txt')))

def all_integers(s):
    """return all integers from a string"""
    return tuple(map(int, re.findall(r'-?\d+', s)))

# Day 20

In [2]:
testcase = open('input\\20-test.txt').read()
testcase[:30]

'Tile 2311:\n..##.#..#.\n##..#...'

In [189]:
testtile = """..#
*.#
#.#"""

testtile = testtile.split('\n')
testtile

['..#', '*.#', '#.#']

In [23]:
def top_side(tile): return tile[0]
def bottom_side(tile): return tile[-1]
def left_side(tile): return ''.join(l[0] for l in tile)
def right_side(tile): return ''.join(l[-1] for l in tile)

top_side(testtile), bottom_side(testtile), left_side(testtile), right_side(testtile)

('..#', '#.#', '.*#', '###')

In [8]:
def get_edges(tile):
    """yield the edges (and reversed egdes) of tile"""
    for op in [top_side, bottom_side, left_side, right_side]:
        yield op(tile)
        yield op(tile)[::-1]
        
list(get_edges(testtile))

['..##.#..#.',
 '.#..#.##..',
 '..###..###',
 '###..###..',
 '.#####..#.',
 '.#..#####.',
 '...#.##..#',
 '#..##.#...']

In [9]:
def parse_tiles(l):
    tiles = {}
    edges = defaultdict(set)
    
    for tile in l.split('\n\n'):
        header, *tile = tile.split('\n')
        assert header[:4] == 'Tile'
        tile_id = int(re.findall(r'-?\d+', header)[0])
        tiles[tile_id] = tile
        for edge in get_edges(tile):
            edges[edge].add(tile_id)
        
    return tiles, edges

tiles, edge_dict = parse_tiles(testcase)
#parse_tiles(testcase)

In [10]:
def find_neighbours(tiles, edge_dict):
    tree = defaultdict(set)
    for tile_id, tile in tiles.items():
        for edge in get_edges(tile):
            for other_tile_id in edge_dict[edge]:
                if other_tile_id == tile_id:
                    continue
                else: 
                    tree[tile_id].add(other_tile_id)
    return tree

find_neighbours(tiles, edge_dict)

defaultdict(set,
            {2311: {1427, 1951, 3079},
             1951: {2311, 2729},
             1171: {1489, 2473},
             1427: {1489, 2311, 2473, 2729},
             1489: {1171, 1427, 2971},
             2473: {1171, 1427, 3079},
             2971: {1489, 2729},
             2729: {1427, 1951, 2971},
             3079: {2311, 2473}})

In [11]:
def partA(l):
    tiles, edge_dict = parse_tiles(l)
    edges = find_neighbours(tiles, edge_dict)
    corners = [t for t, edges in edges.items() if len(edges) == 2]
    
    return reduce(lambda x,y: x*y, corners)

partA(testcase)

20899048083289

In [12]:
inp = open('input\\20.txt').read()
partA(inp)

66020135789767

# part B

In [32]:
def rot90(tile):
    """rotate a tile 90 deg CCW: https://stackoverflow.com/a/53251028/4965175"""
    return [''.join(tile[j][i] for j in range(len(tile))) for i in range(len(tile[0])-1,-1,-1)]

testtile, rot90(testtile), rot90(rot90(testtile)), rot90(rot90(rot90(testtile))), rot90(rot90(rot90(rot90(testtile))))

(['..#', '*.#', '#.#'],
 ['###', '...', '.*#'],
 ['#.#', '#.*', '#..'],
 ['#*.', '...', '###'],
 ['..#', '*.#', '#.#'])

In [191]:
def flip(tile):
    """diagonally flip tile (turn tile over)"""
    return [''.join(tile[j][i] for j in range(len(tile))) for i in range(len(tile[0]))]
testtile, filp(testtile), flip(flip(testtile))

(['..#', '*.#', '#.#'], ['.*#', '...', '###'], ['..#', '*.#', '#.#'])

In [51]:
def pprint_tile(tile):
    rmax = len(tile[0])
    cmax = len(tile)
    for r in range(rmax):
        for c in range(cmax):
            print(tile[r][c], end='')
        print('')
pprint_tile(testtile)

..#
*.#
#.#


In [52]:
def gen_tile_permutations(tile):
    """yield all possible tile placements (8 per tile)"""
    
    yield tile
    for op in [rot90, rot90, rot90, flip, rot90, rot90, rot90]:
        tile = op(tile)
        yield tile
        
for tile in list(gen_tile_permutations(testtile)):
    pprint_tile(tile)
    print()

..#
*.#
#.#

###
...
.*#

#.#
#.*
#..

#*.
...
###

#.#
*.#
..#

###
...
#*.

#..
#.*
#.#

.*#
...
###



In [69]:
def get_tile_id(tile_set, mytile_id):
    """return the neighbouring tile id"""
    tile_set ^= {mytile_id}  # remove mytile
    assert len(tile_set) == 1
    return tile_set.pop()

get_tile_id({9999, 1234}, 1234)

9999

In [179]:
def cut_edges(tile):
    """return tile without edges"""
    return [''.join(tile[i][j] for j in range(1, len(tile)-1)) for i in range(1, len(tile[0])-1)]

cut_edges(testtile)

['###.#...',
 '.#.#....',
 '...#..#.',
 '##..##.#',
 '#.####..',
 '###.#.#.',
 '#.####..',
 '#..#.##.']

In [124]:
def place_cornertile(cornertile_id, tiles, edges, edge_dict):
    for cornertile in gen_tile_permutations(tiles[cornertile_id]):
        # try all possible ways to lay the corners stone, it fits when both
        #  bottomside and rightside fit.
        if len(edge_dict[bottom_side(cornertile)]) == 2 and len(edge_dict[right_side(cornertile)]) == 2:
            # select this one
            selected_tile = cornertile
            selected_tile_id = cornertile_id
            #print('below corner: ', get_tile_id(edge_dict[bottom_side(cornertile)], cornertile_id))
            #print('rightside', get_tile_id(edge_dict[right_side(cornertile)], cornertile_id))
            return selected_tile_id, selected_tile

def match_tile(edge, edge_func, tiles):
    """find tile that matches `edge` on the side that `edge_func` selects"""
    for tile_id in tiles.keys():
        for tile in gen_tile_permutations(tiles[tile_id]):
            if edge == edge_func(tile):
                # select this one
                selected_tile = tile
                selected_tile_id = tile_id
                #print('found', tile_id)
                return selected_tile_id, selected_tile


In [186]:
def create_grid(l):
    tiles, edge_dict = parse_tiles(l)
    edges = find_neighbours(tiles, edge_dict)
    corners = [t for t, edges in edges.items() if len(edges) == 2]
    
    queue = tiles  # tile not placed
    
    cornertile_id = corners[0]
    print('top left = ', cornertile_id)
    N = int(len(tiles)**.5)
    print(f'{N}x{N} grid')
    
    grid = [[None for _ in range(N)] for _ in range(N)]
    cut_grid = [[None for _ in range(N)] for _ in range(N)]
    grid_id = [[None for _ in range(N)] for _ in range(N)]

    for r in range(N):
        for c in range(N):
            if r == 0 and c == 0:
                selected_tile_id, selected_tile = place_cornertile(cornertile_id, tiles, edges, edge_dict)
                del queue[selected_tile_id]
            elif c == 0:
                # place left-most tile
                selected_tile_id, selected_tile = match_tile(bottom_side(grid[r-1][c]), top_side, queue)
                del queue[selected_tile_id]
            else:
                # place tiles from left to right
                selected_tile_id, selected_tile = match_tile(right_side(grid[r][c-1]), left_side, queue)
                del queue[selected_tile_id]
            grid[r][c] = selected_tile
            grid_id[r][c] = selected_tile_id     
            cut_grid[r][c] = cut_edges(selected_tile)
            
    assert queue == {}  # really done?
    
    #print(grid)
    print(grid_id)
    
    return [''.join(cut_grid[row][c][r] for c in range(N)) for row in range(N) for r in range(8)]

create_grid(testcase)

top left =  1951
3x3 grid
[[1951, 2729, 2971], [2311, 1427, 1489], [3079, 2473, 1171]]


['.####...#####..#...###..',
 '#####..#..#.#.####..#.#.',
 '.#.#...#.###...#.##.##..',
 '#.#.##.###.#.##.##.#####',
 '..##.###.####..#.####.##',
 '...#.#..##.##...#..#..##',
 '#.##.#..#.#..#..##.#.#..',
 '.###.##.....#...###.#...',
 '#.####.#.#....##.#..#.#.',
 '##...#..#....#..#...####',
 '..#.##...###..#.#####..#',
 '....#.##.#.#####....#...',
 '..##.##.###.....#.##..#.',
 '#...#...###..####....##.',
 '.#.##...#.##.#.#.###...#',
 '#.###.#..####...##..#...',
 '#.###...#.##...#.######.',
 '.###.###.#######..#####.',
 '..##.#..#..#.#######.###',
 '#.#..##.########..#..##.',
 '#.#####..#.#...##..#....',
 '#....##..#.#########..##',
 '#...#.....#..##...###.##',
 '#..###....##.#...##.##.#']

In [192]:
#create_grid(inp)

In [257]:
def get_monster_pattern():
    monster_pattern = \
"""                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.split('\n')
        
    return [(r, c) for r in range(len(monster_pattern)) for c in range(len(monster_pattern[0])) if monster_pattern[r][c] == '#']

test = \
"""####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.O#..
#.O.##.OO#.#.OO.##.OOO##
..#O.#O#.O##O..O.#O##.##"""

test = test.split('\n')

def count_monsters(grid, monster_char='#'):
    monster_pattern = get_monster_pattern()
    n = 0
    for r in range(len(grid) - 2):
        for c in range(len(grid[0]) - 20):
            if all(grid[r + dr][c + dc] == monster_char for dr, dc in monster_pattern):
                print('Found monster: ', r, c)
                n += 1
    return n

count_monsters(test, 'O')


Found monster:  2 2


1

In [258]:
grid = create_grid(testcase)

top left =  1951
3x3 grid
[[1951, 2729, 2971], [2311, 1427, 1489], [3079, 2473, 1171]]


In [259]:
def partB(l):
    grid = create_grid(l)
    n_monsters = sum(count_monsters(gridpermutation) for gridpermutation in gen_tile_permutations(grid))
    # assume monsters do not overlap (please!!!)
    return sum(grid[r][c] == '#' for r in range(len(grid)) for c in range(len(grid[0]))) - n_monsters * len(get_monster_pattern())

partB(testcase)

top left =  1951
3x3 grid
[[1951, 2729, 2971], [2311, 1427, 1489], [3079, 2473, 1171]]
Found monster:  2 2
Found monster:  16 1


273

In [260]:
partB(inp)

top left =  2383
12x12 grid
[[2383, 3779, 2503, 3847, 2063, 2833, 3049, 2239, 3593, 2663, 2039, 3881], [2287, 2411, 1987, 3853, 2251, 1283, 1487, 1879, 2099, 1367, 1427, 1019], [3541, 1289, 1867, 2797, 1163, 3169, 2003, 3671, 3833, 3947, 1091, 1747], [1307, 2659, 3109, 3041, 2087, 1381, 3181, 3877, 1009, 2179, 3323, 1291], [1823, 2237, 3719, 2819, 2707, 2671, 2971, 1423, 3547, 1699, 1049, 1709], [1741, 1069, 1753, 3527, 3709, 1787, 1609, 1319, 1559, 2749, 1201, 1499], [1097, 3203, 2791, 2687, 1907, 2549, 3329, 1303, 1873, 1021, 2267, 3761], [2129, 2731, 1801, 3137, 1627, 2953, 1597, 1399, 1783, 3499, 3389, 2213], [1721, 2027, 1667, 1619, 3797, 3167, 2467, 3533, 2393, 1453, 3457, 1171], [1279, 2609, 3299, 2633, 1637, 2999, 3433, 1543, 1997, 1483, 2459, 2011], [2357, 2389, 2297, 3413, 1213, 3989, 2683, 2381, 3701, 1433, 2017, 3889], [2593, 3907, 1663, 2273, 2711, 3319, 2113, 2309, 1583, 3343, 2539, 2753]]
Found monster:  2 7
Found monster:  3 72
Found monster:  5 38
Found monster:  13 6


1537